<a href="https://colab.research.google.com/github/Kenjiro1/Formacao_IA/blob/main/Projetos/Engenharia_de_Atributos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>